# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from azureml.data.dataset_factory import TabularDatasetFactory
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [3]:
#Testing the authentication using the Workspace method "from_config"
Workspace.from_config()

Workspace.create(name='quick-starts-ws-137889', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-137889')

In [4]:
#create workspace
ws = Workspace.from_config()

#original code:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'your experiment name here'

experiment=Experiment(ws, experiment_name)

In [5]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment-1,quick-starts-ws-137889,Link to Azure Machine Learning studio,Link to Documentation


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:


dataset = TabularDatasetFactory.from_delimited_files('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')


dataset = dataset.to_pandas_dataframe()
dataset.describe()

,pclass,survived,sibsp,parch,fare
count,1309.000000,1309.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,0.498854,0.385027,33.295479
std,0.837836,0.486055,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,8.000000,9.000000,512.329200


In [7]:
###Just in case:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


### Clean Data:

In [8]:
from train import clean_data

In [9]:
x, y = clean_data(dataset)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
# create scaler
variables = x.columns.tolist()

scaler = StandardScaler()
scaler.fit(x[variables]) 

x = scaler.transform(x[variables])

In [12]:
x = pd.DataFrame(x,columns=variables)


In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.2,
                                                    random_state=0)

In [14]:
#bring them together them again
dataset = pd.concat([x_train,y_train],axis=1)

In [15]:
dataset.head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
1118,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
44,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
1072,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
1130,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
574,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


In [16]:
#To train the model we need a TabularDataset and not a dataframe, therefore the current df will be converterd 
#into a TabularDataset:

#Convert the dataframe into a csv
local_path = 'prepared.csv'

#Save it locally
dataset.to_csv(local_path,index=None)

#Generate the a datastore object which is the the default datastore
datastore = ws.get_default_datastore()

In [17]:
#Upload the dataframe which was previosly converted into a csv
datastore.upload(src_dir='.', target_path='data')

Uploading an estimated of 21 files
Target already exists. Skipping upload for data/.amlignore
Target already exists. Skipping upload for data/.amlignore.amltmp
Target already exists. Skipping upload for data/automl3.ipynb
Target already exists. Skipping upload for data/automl3.ipynb.amltmp
Target already exists. Skipping upload for data/hyperparameter_tuning.ipynb
Target already exists. Skipping upload for data/prepared.csv
Target already exists. Skipping upload for data/pre_preprocess.py
Target already exists. Skipping upload for data/score.py
Target already exists. Skipping upload for data/score.py.amltmp
Target already exists. Skipping upload for data/train.py
Target already exists. Skipping upload for data/.ipynb_aml_checkpoints/automl3-checkpoint2021-1-7-15-32-48.ipynb
Target already exists. Skipping upload for data/__pycache__/pre_preprocess.cpython-36.pyc
Target already exists. Skipping upload for data/__pycache__/train.cpython-36.pyc
Uploading ./automl.log
Uploaded ./automl.log

$AZUREML_DATAREFERENCE_1dceef11c58048109999751130ae8ec5

In [18]:
#For the sake of checking; check the path
datastore.path()

$AZUREML_DATAREFERENCE_workspaceblobstore

In [19]:
#Now the uploaded file will be transformed into a Tabular dataset and store in a varible named 'training_dataset'
training_dataset = Dataset.Tabular.from_delimited_files(path= [(datastore,('data/prepared.csv'))])

In [20]:
#let's visualize the data:
training_dataset.to_pandas_dataframe().head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
0,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
1,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
2,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
3,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
4,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [21]:
#Create the automl settings which will be used as argurments in the automl config
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

#Create the automl_config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=training_dataset,
                             label_column_name="survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [22]:
#New one
#Original code
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
automl_run = experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_198f108e-08a7-4ea8-b191-e589efb0305d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:       

In [23]:
#Additional Run Details
from azureml.widgets import RunDetails

RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
#Get generic outputs from the automl_run
automl_run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


(Run(Experiment: automl-experiment-1,
 Id: AutoML_198f108e-08a7-4ea8-b191-e589efb0305d_71,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.5208333333333334,
                                                                                                scale_pos_weight=1,
 

In [25]:
#Get the best model outputs
best_automl_run, best_model = automl_run.get_output()


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [26]:
#Get the best model id
print(best_automl_run.id)

AutoML_198f108e-08a7-4ea8-b191-e589efb0305d_71


In [27]:
os.makedirs('outputs',exist_ok=True)

In [28]:
#My support
best_automl_run.download_file('outputs/model.pkl', 'outputs/best_run_automl_2.pkl')
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_automl_run.download_file('automl_driver.py', 'outputs/automl_driver.py')

In [29]:
#let's deploy since the automl is stronger thatn the hyperparametr model
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


In [30]:
#Register your best model
model = best_automl_run.register_model(model_name = "best_run_automl_2.pkl", model_path = './outputs/') 

#model = Model.register(workspace = ws,
 #                       model_path ='./outputs/' ,
  #                      model_name = "best_run_automl.pkl") Este era el de Ulises

#model_path = '/odl_user_137436/outputs'
#model = Model.register(workspace= ws, model_name = "best_run_automl_2.pkl", model_path = './outputs/')

#~/cloudfiles/code/Users/odl_user_137436/outputs

In [31]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-137889', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-137889'), name=best_run_automl_2.pkl, id=best_run_automl_2.pkl:2, version=2, tags={}, properties={})

In [32]:
print(model.id)


best_run_automl_2.pkl:2


In [33]:
#Defining the deployment
from azureml.core.webservice import AciWebservice

In [34]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                              enable_app_insights=True)

In [35]:
#Generating the environment needed:
env = best_automl_run.get_environment() # Este es elviejo

#env = Environment.from_conda_specification(name='myenv',file_path = 'env.yml') 

inference_config = InferenceConfig(entry_script='./score.py',
                                    environment=env)



In [36]:
f = open("env.yml", "w")
f.write(env.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", env.python.conda_dependencies.serialize_to_string())

packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.21.0.post1
  - azureml-pipeline-core==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-defaults==1.21.0
  - azureml-interpret==1.21.0
  - azureml-automl-core==1.21.0
  - azureml-automl-runtime==1.21.0
  - azureml-train-automl-client==1.21.0
  - azureml-train-automl-runtime==1.21.0
  - azureml-dataset-runtime==1.21.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_20a8278aa8b20dd48cc50f56a6d2586c



In [40]:
service_name = 'my-service-ml-2'

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config)

In [41]:
#check on service
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-137889', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-137889'), name=my-service-ml-2, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [42]:
#Check on deployment status:
service.wait_for_deployment(True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 165cb02b-2878-44b8-98ea-ccc776506334
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/auto-ml/code/Users/odl_user_137889/outputs/best_run_automl_2.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/auto-ml/code/Users/odl_user_137889/outputs/best_run_automl_2.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 165cb02b-2878-44b8-98ea-ccc776506334
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/auto-ml/code/Users/odl_user_137889/outputs/best_run_automl_2.pkl', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, FileNotFoundError: [Errno 2] No such file or directory: '/mnt/batch/tasks/shared/LS_root/mounts/clusters/auto-ml/code/Users/odl_user_137889/outputs/best_run_automl_2.pkl', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": 

In [43]:
print(service.get_logs())

2021-02-07T17:04:33,002249109+00:00 - iot-server/run 
2021-02-07T17:04:33,002662211+00:00 - rsyslog/run 
2021-02-07T17:04:33,003574915+00:00 - nginx/run 
2021-02-07T17:04:33,003869316+00:00 - gunicorn/run 
rsyslogd: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:
#Now let's get the scoring URI:
scoring_uri = service.scoring_uri
scoring_uri

In [ ]:
#let's test:
import json
import requests

scoring_uri = scoring_uri
headers = {'Content-Type':'application/json'}



test_data = json.dumps({'data':[{
    'pclass': 0.8419164182590155,
    'age': -0.34907541344456255,
    'sibsp': -0.47908676070718687,
    'parch': -0.444999501816175,
    'fare': -0.4902404567566683,
    'age_NA': -0.5014319838391105,
    'fare_NA': -0.027650063180466557,
    'sex_male': 0.743496915331831,
    'cabin_Missing': 0.5393765119990418,
    'cabin_Rare': -0.42592011250734235,
    'embarked_Q': -0.32204029159373954,
    'embarked_Rare': -0.03911805059269843,
    'embarked_S': 0.6573935670276714,
    'title_Mr': 0.8525918887485938,
    'title_Mrs': -0.42592011250734235,
    'title_Rare': -0.27494677157229536
    }
    ]
        })

test_data2 = json.dumps({'data':[{
    'pclass': -15460978645168200,
    'age': 0.8912042887450313,
    'sibsp': -0.47908676070718687,
    'parch': -0.444999501816175,
    'fare': 19569900306355100,
    'age_NA': -0.5014319838391105,
    'fare_NA': -0.027650063180466557,
    'sex_male': -13449954927569300,
    'cabin_Missing': -18539924853119600,
    'cabin_Rare': 23478581326275300,
    'embarked_Q': -0.32204029159373954,
    'embarked_Rare': -0.03911805059269843,
    'embarked_S': -15211587854766800,
    'title_Mr': -11728941046668400,
    'title_Mrs': -0.42592011250734235,
    'title_Rare': -0.27494677157229536

    }
    ]
        })


response = requests.post(scoring_uri, data=test_data2, headers=headers)

print("Result:",response.text)